In [3]:
import Levenshtein
import requests

from tqdm import tqdm
from statistics import mean

from extract_books_from_DB import *

In [2]:
all_books = generate_all_books()

ADP_loading_time:  10.358738422393799
DL_loading_time:  42.671600580215454
ILE_loading time:  14.520714044570923
loading_data_time:  67.851145029068
recoupement des auteurs
{'total': 21515, 'ADP': 5201, 'ILE': 0, 'Hurtubise': 884, 'Babelio': 1651, 'Depot_legal': 13779}


100%|██████████| 102985/102985 [00:00<00:00, 122249.71it/s]


In [ ]:
author_ls = {}

for book in tqdm(all_books, total=len(all_books)):
    for author in book.authors:
        if author not in author_ls:
            author_ls[author] = [book]
        else:
            author_ls[author].append(book)

with open('authors_ls.json', 'w') as outfile:
    json.dump(author_ls, outfile, cls=BookJSONEncoder)

raw_author_ls = {}

for book in tqdm(all_books, total=len(all_books)):
    for author in book.authors_raw:
        if author not in author_ls:
            author_ls[author] = [book]
        else:
            author_ls[author].append(book)

with open('raw_authors_ls.json', 'w') as outfile:
    json.dump(author_ls, outfile, cls=BookJSONEncoder)

In [4]:
"""
Numero de prop importante:
- sex or gender (P21)
- country of citizenship (P27)
- name in native language (P1559)
- birth name (P1477)
-  given name  (P735)
- family name (P734)
- date of birth (P569)
- place of birth (P19)
- date of death  (P570)
- place of death  (P20)
- occupation (P106)
    - author (Q482980)
    - writer (Q36180)
- notable work (P800)
- genre (P136)
- award received (P166)
- nominated for (P1411)
- country of citizenship (P27)

premiere query: ecrivain canadiens parlant/ecrivant en francais => 1232 resultats
SELECT ?item ?itemLabel
WHERE
{{

    ?item wdt:P31 wd:Q5 .
    ?item wdt:P106 wd:Q36180 .
    ?item wdt:P27 wd:Q16 .
    ?item wdt:P1412 wd:Q150

    SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en".}
}}

deuxieme query: ecrivain canadiens parlant/ecrivant en francais qui ont un produit notable => 82 resultats
SELECT ?item ?itemLabel ?bookLabel
WHERE
{

  ?item wdt:P31 wd:Q5 .
  ?item wdt:P106 wd:Q36180 .
  ?item wdt:P27 wd:Q16 .
  ?item wdt:P1412 wd:Q150 .
  ?item wdt:P800 ?book

  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en".}
}

"""

In [12]:
# On recherche par nom, et par occupation (occupation fille de "ecrivant"), et on en récupère les labels
q = """
SELECT DISTINCT ?s ?book ?bookLabel
WHERE
{{
    ?s wdt:P31 wd:Q5 .
    ?s wdt:P106 ?occup .
    ?occup wdt:P279 wd:Q36180 .

    {{ ?s rdfs:label "{0}"@fr }} UNION {{ ?s rdfs:label "{0}"@en }} UNION {{ ?s skos:altLabel "{0}"@fr }} .
    OPTIONAL {{ ?s wdt:P800 ?book . }}

    SERVICE wikibase:label {{ bd:serviceParam wikibase:language "fr,en".}}
}}
"""

100%|██████████| 11/11 [00:56<00:00,  5.13s/it]


tset query
Nombre d'auteurs trouvés:  9  soit  81.81818181818181 %


In [ ]:
# test with famous authors

famous = ['Michel Tremblay', 'Anne Hébert', 'Gabrielle Roy', 'Marie Cardinal', 'Réjean Ducharme',
                'Jacques Ferron', 'Victor-Lévy Beaulieu', 'Marcel Dubé', 'Yves Thériault', 'Jacques Poulin',
                'André Langevin']

search_results = {}
print('tset query')
nb_author_found = 0
for author in tqdm(famous, total=len(famous)):
    time.sleep(1)
    rep_wikidata_ecrivain = requests.get('https://query.wikidata.org/sparql?format=json&query=' + q.format(author))

    if rep_wikidata_ecrivain.status_code == 429:
        print("trop rapide !!!")

    if rep_wikidata_ecrivain.status_code == 200:
        if len(rep_wikidata_ecrivain.json()['results']['bindings']) > 0:
            nb_author_found += 1
            search_results[author] = {'id':rep_wikidata_ecrivain.json()['results']['bindings'][0]['s']['value'], 'books':[]}
            for result in rep_wikidata_ecrivain.json()['results']['bindings']:
                try:
                    search_results[author]['books'].append({'book_id': result['book']['value'], 'book_label': result['bookLabel']['value']})
                except KeyError:
                    pass
print("Nombre d'auteurs trouvés: ", nb_author_found, " soit ", nb_author_found * 100/len(famous), "%")

In [14]:
# recherche par auteurs

search_results = {}
print('query')
nb_author_found = 0
for author in tqdm(raw_author_ls, total=len(raw_author_ls)):
    time.sleep(1)
    rep_wikidata_ecrivain = requests.get('https://query.wikidata.org/sparql?format=json&query=' + q.format(author))

    if rep_wikidata_ecrivain.status_code == 429:
        print("trop rapide !!!")

    if rep_wikidata_ecrivain.status_code == 200:
        if len(rep_wikidata_ecrivain.json()['results']['bindings']) > 0:
            nb_author_found += 1
            search_results[author] = {'id':rep_wikidata_ecrivain.json()['results']['bindings'][0]['s']['value'], 'books':[]}
            for result in rep_wikidata_ecrivain.json()['results']['bindings']:
                try:
                    search_results[author]['books'].append({'book_id': result['book']['value'], 'book_label': result['bookLabel']['value']})
                except KeyError:
                    pass
print("Nombre d'auteurs trouvés: ", nb_author_found, " soit ", nb_author_found * 100/len(raw_author_ls), "%")

with open('wikidata_author_&_books_found_by_name_search.json', 'w') as outfile:
    json.dump(search_results, outfile)

100%|██████████| 21515/21515 [7:14:56<00:00,  1.21s/it]   


query
trop rapide !!!
trop rapide !!!
trop rapide !!!
trop rapide !!!
trop rapide !!!
trop rapide !!!
trop rapide !!!
trop rapide !!!
trop rapide !!!
Nombre d'auteurs trouvés:  2582  soit  12.000929584011155 %


In [6]:
# On cherche maintenant tout les auteurs francophones de wikidata pour faire la recherche localement
all_authors_query = """
SELECT DISTINCT ?author ?authorLabel ?bookLabel ?awardLabel
WHERE
{
    ?author wdt:P31 wd:Q5 .
    ?author wdt:P27 ?country .
    ?country wdt:P2936 wd:Q150 .
    ?author wdt:P106 ?occup .
    ?occup wdt:P279 wd:Q36180

    OPTIONAL { ?author wdt:P800 ?book . }
    OPTIONAL { ?author wdt:P166 ?award . }

    SERVICE wikibase:label { bd:serviceParam wikibase:language "fr,en".}

}"""
rep_wikidata_ecrivain = requests.get('https://query.wikidata.org/sparql?format=json&query=' + all_authors_query)
print('code_status: ', rep_wikidata_ecrivain.status_code)
if rep_wikidata_ecrivain.status_code == 200:
    if len(rep_wikidata_ecrivain.json()['results']['bindings']) > 0:
        print('success ! Nombre d\e lignes: ', len(rep_wikidata_ecrivain.json()['results']['bindings']))

code_status:  200


In [10]:
# On sauvegarde le résultat de la recherche car elle peut échouer en fonction de la charge sur les serveurs de wikidata
if rep_wikidata_ecrivain.status_code == 200:
    if len(rep_wikidata_ecrivain.json()['results']['bindings']) > 0:
        with open('Wikidata/request_allfrench_author_books_price_wikidata.json', 'w') as output_file:
            json.dump(rep_wikidata_ecrivain.json()['results']['bindings'], output_file)

In [6]:
with open('Wikidata/request_allfrench_author_books_price_wikidata.json') as input_file:
    res = json.load(input_file)

cleaned_data = {}

for result in res:
    try:
        cleaned_data[result['author']]['works'].add(result['bookLabel'])
        cleaned_data[result['author']]['awards'].add(result['awardLabel'])
    except KeyError:
        cleaned_data[result['author']] = {
            'author_uri': result['author'],
            'author_label': result['authorLabel'],
            'works': {[result['bookLabel']]} if 'bookLable' in result else set(),
            'awards': {result['awardLabel']} if 'awardLabel' in result else set()
        }

with open('Wikidata/parsed_request_allfrench_author_books_price_wikidata.json', 'w') as output_file:
    json.dump(cleaned_data, output_file, cls=BookJSONEncoder)

In [9]:
# Comparaison avec wikidata
with open('Wikidata/parsed_request_allfrench_author_books_price_wikidata.json', 'r') as author_file:
    wikidata_authors = json.load(author_file)

with open('authors_ls.json', 'r') as input_file:
    author_ls = json.load(input_file, cls=BookJSONDecoder)

# Couple dont on estime que les deux auteurs sont les mêmes
couple_writers = []
# Couple dont on estime que les deux auteurs sont différents mais à la limite d'être les mêmes
neg_couple_writers = []

# Compte les livres des auteurs par base de donnée (permet de voir si une base de donnée n'a aucun match)
data_base_stats = {}
# Compte les auteurs de wikidata qui ont été trouvé dans nos bases de donnée
found_at_least_once = 0
# Compte le nombre de couples
nb_couple = 0
# Compte le nombre de fois qu'un auteur de wikidata a été trouvé dans nos bases de donnée
author_found_freq = {}

for wiki_author_id, wiki_author_info in tqdm(wikidata_authors.items()):
    normalised_wikidata_title = normalize(wiki_author_info['author_label'])
    # Compte le nombre de fois que l'auteur wikidata a été associé a un auteur de nos bases de donnée
    wiki_author_found_count = 0
    for DB_author, DB_author_books in author_ls.items():
        dist_auteur = Levenshtein.distance(DB_author, normalised_wikidata_title)
        if max(1, min(len(DB_author), len(normalised_wikidata_title)) / 6) > dist_auteur > max(1, min(len(DB_author), len(normalised_wikidata_title)) / 8):
            neg_couple_writers.append({
                        'author_DB': {'name': DB_author, 'books': DB_author_books},
                        'author_wiki': {'name': normalised_wikidata_title,
                                        'name_raw': wiki_author_info['author_label'],
                                        'info': wiki_author_info}
                        })
        elif dist_auteur < max(1, min(len(DB_author), len(normalised_wikidata_title)) / 8):

            new_couple = {
                        'author_DB': {'name': DB_author, 'books': DB_author_books},
                        'author_wiki': {'name': normalised_wikidata_title,
                                        'name_raw': wiki_author_info['author_label'],
                                        'info': wiki_author_info}
                    }

            nb_couple += 1
            wiki_author_found_count += 1

            couple_writers.append(new_couple)
            for book in DB_author_books:
                try:
                    data_base_stats[book['data_base']] += 1
                except:
                    data_base_stats[book['data_base']] = 1

    if wiki_author_found_count:
        found_at_least_once += 1
        author_found_freq[wiki_author_info['author_label']] = wiki_author_found_count

print("nombre d'autheurs en commun: ", found_at_least_once, " soit ", found_at_least_once * 100 / len(list(author_ls)), "%")
print("nombre d'autheurs reconnus: ", nb_couple, " soit ", nb_couple/found_at_least_once, ' auteur dans nos bases de donnée par auteur reconnu dans wikidata')
print("Frequence de reconnaissance des 10 auteurs de wikidata les plus reconnus: ", sorted(author_found_freq.items(), key=lambda item: item[1], reverse=True)[:10])


print("Répartition des livres des auteurs reconnus dans les bases de données: ")
print('1/8: ', json.dumps(data_base_stats))

print("20 premiers couples à la limite d'être considérés comme égaux")
for couple in neg_couple_writers[:20]:
    print("Autheur DB: ", couple['author_DB']['name'] + " " * (40 - len(couple['author_DB']['name'])),
          "Autheur wikidata: ", couple['author_wiki']['name'])

with open('Wikidata/couple_authors_wikidata.json', 'w') as outfile:
    json.dump(couple_writers, outfile, cls=BookJSONEncoder)

100%|██████████| 18826/18826 [07:38<00:00, 41.09it/s]


nombre d'autheurs en commun:  1277  soit  6.544019678179768 %
nombre d'autheurs reconnus:  1345  soit  1.0532498042286609  auteur dans nos bases de donnée par auteur reconnu dans wikidata
Frequence de reconnaissance des 10 auteurs de wikidata les plus reconnus:  [('Michel Côté', 3), ('Denise Boucher', 3), ('Michel Tremblay', 3), ('Sylvie Bérard', 3), ('Jean-Claude Lauzon', 2), ('Michel Marc Bouchard', 2), ('Denis Thériault', 2), ('Gaëtan Brulotte', 2), ('Jacques Godbout', 2), ('Francis Simard', 2)]
Répartition des livres des auteurs reconnus dans les bases de données: 
1/8:  {"ILE": 13478, "Depot_legal": 8764, "ADP": 2681, "Babelio": 629}
20 premiers couples à la limite d'être considérés comme égaux
Autheur DB:  david descoteaux                         Autheur wikidata:  david decoteau
Autheur DB:  pierre lambert                           Autheur wikidata:  pierre aubert
Autheur DB:  pierre hebert                            Autheur wikidata:  pierre aubert
Autheur DB:  etienne poirier 

In [13]:
count_books = 0
nb_books_for_each_wiki_author = []

count_awards = 0
nb_awards_for_each_wiki_author = []

all_awards_name = set()

for couple in couple_writers:
    if couple['info_wikidata']['works']:
        count_books += 1

    if couple['info_wikidata']['awards']:
        count_awards += 1
        all_awards_name.update(couple['info_wikidata']['awards'])

    nb_books_for_each_wiki_author.append(len(couple['info_wikidata']['works']))
    nb_awards_for_each_wiki_author.append(len(couple['info_wikidata']['awards']))
print("Nombre d'artiste ayant des livres attribués: ", count_books)
print("Nombre de livres global récupérés: ", sum(nb_books_for_each_wiki_author))

print("Nombre d'artiste ayant des prix attribués: ", count_awards)
print("Nombre de prix global récupérés: ", sum(nb_awards_for_each_wiki_author))

print("exemple des prix: ", all_awards_name[:20])

Nombre d'artiste ayant des livres attribués:  61
Nombre de livres global récupérés:  193
Nombre d'artiste ayant des prix attribués:  414
Nombre de prix global récupérés:  1117
exemple des prix:  {'Golden Globe de la meilleure actrice dans un film dramatique', "Ours d'or", 'prix Booker', 'Panthéon de la musique canadienne', 'prix Schiller', "prix de poésie de la ville d'Angers", 'prix Mercier', 'prix Sant Jordi du cinéma', 'Prix Gilles-Corbeil', 'Prix de la bande à Mœbius', 'Prix des lecteurs du Marché de la poésie de Montréal', 'Prix du grand public du Salon du livre de Montréal - La Presse', 'PEN/Nabokov Award', 'Prix des Écrivains francophones d’Amérique', 'prix Denise-Pelletier', 'Aurora Award for Best Graphic Novel', 'Gerald Lampert Award', 'prix international Man-Booker', 'prix du scénario du Festival de Cannes', 'prix Tollander', 'Neustadt International Prize for Literature', 'Zamenhof award', 'prix Anthony', 'Stella Prize', 'prix Viareggio', 'Prix littéraire Adrienne-Choquette',

In [32]:
# Listes pour stoker nos couples confirmé, invalidé et ceux dont on ne peut pas conclure car il n'y a pas de titres
couples_confirmed = []
couples_infirmed = []
couples_unsure = []
# Compte les cas ci-dessus
count_checked_by_titles = 0
count_no_titles = 0
count_differentiate_by_titles = 0

for couple in tqdm(couple_writers):
    confirmed_by_title_match = False
    confirmed_by_title_match_re = False

    # On vérifie si on peut trouver des titres proches en comparant les titres parsés
    # au titres normalisés de nos bases de donnée
    book_titles_DB = [book['title'] for book in couple['book DB']]
    for title in book_titles_DB:
        normalised_wikidata_title_ls = [normalize(title_wikidata) for title_wikidata in couple['info_wikidata']['works']]
        for title_wikidata in normalised_wikidata_title_ls:
            dist_titles = Levenshtein.distance(title, title_wikidata)
            if dist_titles < max(1, min(len(title), len(title_wikidata)) / 4):
                confirmed_by_title_match = True
                break
    # On recherche par le suite le titre du livre directement dans les titres non nettoyé de la page wikipédia
    # via expression régulières
    for title in book_titles_DB:
        for title_wiki_raw in couple['info_wikidata']['works']:
            if re.search(normalize(title), normalize(title_wiki_raw)):
                confirmed_by_title_match_re = True
                break

    # On liste les cas possibles: 0,1,2 tests réussis
    if (not confirmed_by_title_match) and (not confirmed_by_title_match_re):
        if couple['info_wikidata']['works']:
            count_differentiate_by_titles += 1
            couples_infirmed.append(couple)
        else:
            count_no_titles += 1
            couples_unsure.append(couple)
    else:
        count_checked_by_titles += 1
        couples_confirmed.append(couple)

    # On sauvegarde comment avons nous trouvé les correspondances entre les deux listes de titres
    couple['confirmed_by_title_match'] = confirmed_by_title_match
    couple['confirmed_by_title_match_re'] = confirmed_by_title_match_re

print('nombre de couple: ', len(couple_writers))
print('nombre de couples confirmés par titres: ', count_checked_by_titles)
print('nombre de couples infirmés par titres: ', count_differentiate_by_titles)
print('nombre de couples sans titres de wikipedia: ', count_no_titles)

with open('Wikidata/couple_authors_wikidata_confirmed.json', 'w') as outfile:
    json.dump(couples_confirmed, outfile, cls=BookJSONEncoder)

with open('Wikidata/couple_authors_wikidata_infirmed.json', 'w') as outfile:
    json.dump(couples_infirmed, outfile, cls=BookJSONEncoder)

with open('Wikidata/couple_authors_wikidata_unsure.json', 'w') as outfile:
    json.dump(couples_unsure, outfile, cls=BookJSONEncoder)

100%|██████████| 1357/1357 [00:00<00:00, 17553.61it/s]


nombre de couple:  1357
nombre de couples confirmés par titres:  28
nombre de couples infirmés par titres:  33
nombre de couples sans titres de wikipedia:  1296


In [25]:
count_awards = 0
nb_awards_for_each_wiki_author = []
for couple in couple_writers:
    if couple['info_wikidata']['awards']:
        count_awards += 1
    nb_awards_for_each_wiki_author.append(len(couple['info_wikidata']['awards']))

print("Nombre d'artiste ayant des prix attribués: ", count_awards)
print("Nombre de prix global récupérés: ", sum(nb_awards_for_each_wiki_author))

Nombre d'auteur dont on a récupéré des prix:  414
Nombre total de prix:  1117


In [7]:
# Recherche de tout les champs disponibles depuis wikidata soit depuis tout les auteurs francophones,
# soit depuis les auteurs dont on a trouvé une instance dans notre base de donnée

# Recherche des URI de tout les auteurs de wikidata francophones
all_authors_query = """
SELECT DISTINCT ?author
WHERE
{
    ?author wdt:P31 wd:Q5 .
    ?author wdt:P27 ?country .
    ?country wdt:P2936 wd:Q150 .
    ?author wdt:P106 ?occup .
    ?occup wdt:P279 wd:Q36180

}"""
rep_wikidata_ecrivain = requests.get('https://query.wikidata.org/sparql?format=json&query=' + all_authors_query)
print('code_status: ', rep_wikidata_ecrivain.status_code)

authors_uri = []
if rep_wikidata_ecrivain.status_code == 200:
    if len(rep_wikidata_ecrivain.json()['results']['bindings']) > 0:
        print('exemples: ')
        print(json.dumps(rep_wikidata_ecrivain.json()['results']['bindings'][:2], indent=2))
        for result in tqdm(rep_wikidata_ecrivain.json()['results']['bindings']):
            authors_uri.append(result['author']['value'])

100%|██████████| 18826/18826 [00:00<00:00, 649352.94it/s]


code_status:  200
exemples: 
[
  {
    "author": {
      "type": "uri",
      "value": "http://www.wikidata.org/entity/Q34922"
    }
  },
  {
    "author": {
      "type": "uri",
      "value": "http://www.wikidata.org/entity/Q35109"
    }
  }
]


In [14]:
result = {}
# # Version pour tout les auteurs francophones
# for uri in tqdm(authors_uri):
# Version pour tout les auteurs dont on a trouvé un équivalent dans notre base de donnée
for couple in tqdm(couple_writers):
    uri = couple['info_wikidata']['author_uri'].split('/')[-1]
    # partie générique
    time.sleep(1)
    get_all_fields_query = """
    SELECT ?pLabel ?objLabel
    WHERE {{
        wd:{0} ?predicate ?obj.
        BIND( IRI(REPLACE( STR(?predicate),"prop/direct/","entity/" )) AS ?p)
        SERVICE wikibase:label {{ bd:serviceParam wikibase:language "fr, en". }}
    }}
    """
    rep_wikidata_ecrivain = requests.get('https://query.wikidata.org/sparql?format=json&query=' + get_all_fields_query.format(uri))

    if rep_wikidata_ecrivain.status_code == 429:
            print("trop rapide !!!")

    if rep_wikidata_ecrivain.status_code == 200:
        if len(rep_wikidata_ecrivain.json()['results']['bindings']) > 0:

            result[uri] = [(duo['pLabel']['value'], duo['objLabel']['value']) for duo in rep_wikidata_ecrivain.json()['results']['bindings']]
    else:
        print("erreur code: ", rep_wikidata_ecrivain.status_code)

100%|██████████| 1357/1357 [30:06<00:00,  1.33s/it]


In [18]:
# Toutes les propriété possibles
all_properties = set()
for properties in result.values():
    all_properties.update([property[0]for property in properties])

# Pour chaque auteur, on compte combien de fois chaque propriété apparait
properties_dict = {}
for author_uri, properties in result.items():
    for knowed_property in all_properties:
        counter = 0
        for property in properties:
            if property[0] == knowed_property:
                counter += 1
        try:
            properties_dict[knowed_property].append(counter)
        except:
            properties_dict[knowed_property] = [counter]

# On tire des mesures du dictionnaire créé si desssus
properties_resume = {}
for knowed_property in properties_dict:
    properties_resume[knowed_property] = {
        # Moyenne du nombre de fois qu'apparait une propriété quand elle est présente chez un auteur
        'non_null_mean': mean([e for e in properties_dict[knowed_property] if e]),
        # Nombres d'auteur qui ont cette propriété
        'nb_non_null': sum([1 for e in properties_dict[knowed_property] if e]),
        # Nombre total de fois qu'apparait une propriété
        'sum': sum(properties_dict[knowed_property]),
    }

# On sauvegarde ces mesures sous csv
with open('Wikidata/list_properies_by_author.csv', 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=['name', 'non_null_mean', 'nb_non_null', 'sum' ])
        writer.writeheader()
        for rep_prop_name, rep_prop_dict in properties_resume.items():
            if rep_prop_name.find("http://www.wikidata.org/prop/") == -1:
                rep_prop_dict['name'] = rep_prop_name
                writer.writerow(rep_prop_dict)

